In [1]:
# Ponowne załadowanie środowiska po resecie
import sys
import os
sys.path.append(os.path.abspath("../"))
import pandas as pd
import numpy as np
from utils.categorize_brand import categorize_brand

# Wczytaj dane testowe
df_test = pd.read_csv("../data/sales_ads_test.csv")

# Usuń zbędne kolumny
df_test = df_test.drop(columns=["Data_publikacji_oferty", "ID"])
df_test = df_test[df_test["Waluta"] == "PLN"]
df_test = df_test.drop(columns=["Waluta"])
df_test = df_test.drop(columns=["Wersja_pojazdu", "Generacja_pojazdu", "Emisja_CO2", "Data_pierwszej_rejestracji", "Lokalizacja_oferty"])

# Pierwszy właściciel: nan → 0, reszta → 1
df_test["Pierwszy_wlasciciel"] = df_test["Pierwszy_wlasciciel"].notna().astype(int)

# Kraj pochodzenia: Poland → 1, reszta → 0
df_test["Kraj_pochodzenia"] = (df_test["Kraj_pochodzenia"] == "Poland").astype(int)

# Stan: New → 1, reszta → 0
df_test["Stan"] = (df_test["Stan"] == "New").astype(int)

# Kategoryzacja marki
df_test["Segment_marki"] = df_test["Marka_pojazdu"].apply(categorize_brand)
df_test = pd.get_dummies(df_test, columns=["Segment_marki"], dtype=int)
df_test = df_test.drop(columns=["Marka_pojazdu"])

# Wypełnij medianą wartości numeryczne
df_test["Przebieg_km"].fillna(df_test["Przebieg_km"].median(), inplace=True)
df_test["Rok_produkcji"].fillna(df_test["Rok_produkcji"].median(), inplace=True)
df_test["Moc_KM"].fillna(df_test["Moc_KM"].median(), inplace=True)
df_test["Pojemnosc_cm3"].fillna(df_test["Pojemnosc_cm3"].median(), inplace=True)

# Konwersja kategorii na liczby
df_test["Rodzaj_paliwa"] = df_test["Rodzaj_paliwa"].fillna("unknown").astype("category").cat.codes
df_test["Naped"] = df_test["Naped"].fillna("unknown").astype("category").cat.codes
df_test["Skrzynia_biegow"] = df_test["Skrzynia_biegow"].fillna("unknown").astype("category").cat.codes
df_test["Typ_nadwozia"] = df_test["Typ_nadwozia"].fillna("unknown").astype("category").cat.codes
df_test["Kolor"] = df_test["Kolor"].fillna("unknown").astype("category").cat.codes
df_test["Model_pojazdu"] = df_test["Model_pojazdu"].fillna("unknown").astype("category").cat.codes

# Wyposażenie → liczba opcji
df_test["Liczba_wyposazenia"] = df_test["Wyposazenie"].apply(lambda x: len(str(x).split(", ")))

# Średnie z treningowego (na twardo)
srednie_wyposazenie = {
    "Segment_marki_Budżetowe": 26.25,
    "Segment_marki_Średnie": 27.97,
    "Segment_marki_Premium": 32.11
}

# Funkcja oznaczająca wyposazenie
def oznacz_wyposazenie(row):
    if row.get("Segment_marki_Budżetowe", 0) == 1:
        srednia = srednie_wyposazenie["Segment_marki_Budżetowe"]
    elif row.get("Segment_marki_Średnie", 0) == 1:
        srednia = srednie_wyposazenie["Segment_marki_Średnie"]
    elif row.get("Segment_marki_Premium", 0) == 1:
        srednia = srednie_wyposazenie["Segment_marki_Premium"]
    else:
        return "-"
    return "+" if row["Liczba_wyposazenia"] > srednia else "-"

df_test["Wyposazenie_kategoria"] = df_test.apply(oznacz_wyposazenie, axis=1)
df_test["Wyposazenie_kategoria"] = df_test["Wyposazenie_kategoria"].map({"+": 1, "-": 0})
df_test = df_test.drop(columns=["Wyposazenie"])

# Zapisz gotowy test
df_test.to_csv("../data/cleaned_test.csv", index=False)


/var/folders/02/2thd89_s1ln77brcp9tybl4h0000gn/T/ipykernel_19471/207028031.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test["Przebieg_km"].fillna(df_test["Przebieg_km"].median(), inplace=True)
/var/folders/02/2thd89_s1ln77brcp9tybl4h0000gn/T/ipykernel_19471/207028031.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whic